In [13]:
from lxml import html, etree
import requests
import re
import json
import sys
import pandas as pd
import numpy as np

In [3]:
headers = { 'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'}
url = "https://www.world-airport-codes.com/search/?s=%s"

In [5]:
props = ['Latitude', 'Longitude', 'Time Zone']
def getAirportInfo(airport_code):
    print("Fetching %s data" % airport_code)
    root = html.fromstring(requests.get(url % airport_code, headers = headers).text)
    info_div = root.xpath("//div[@class='airport-info-table']")
    if(len(info_div) < 1):
        return {}
    info_div = info_div[0]
    res = {}
    for prop in props:
        val = info_div.xpath("//span[@data-key='%s']/@data-value" % prop)[0]
        res[prop] = val
    res['Time Zone'] = re.search('\(.*\s(.*)\)', res['Time Zone']).group(1)
    return res

In [10]:
df = pd.read_csv("bfd/train_dataset.csv")
test_df = pd.read_csv("bfd/test_dataset.csv")

In [15]:
data = {}
or_ports = df["Origin"].unique()
ds_ports = df["Dest"].unique()
or_ports_test = test_df["Origin"].unique()
ds_ports_test = test_df["Dest"].unique()
mr_ports = or_ports
mr_ports = np.concatenate((mr_ports, [x for x in ds_ports if x not in mr_ports]))
mr_ports = np.concatenate((mr_ports, [x for x in or_ports_test if x not in mr_ports]))
mr_ports = np.concatenate((mr_ports, [x for x in ds_ports_test if x not in mr_ports]))
print(len(mr_ports))
for c in mr_ports:
    data[c] = getAirportInfo(c)

305
Fetching AUS data
Fetching TVC data
Fetching BDL data
Fetching XNA data
Fetching PHX data
Fetching CLT data
Fetching GEG data
Fetching LAS data
Fetching DEN data
Fetching CAK data
Fetching HNL data
Fetching SFO data
Fetching MSP data
Fetching BHM data
Fetching PHL data
Fetching TYS data
Fetching ONT data
Fetching LGA data
Fetching OAK data
Fetching LGB data
Fetching BOS data
Fetching MEM data
Fetching STL data
Fetching DFW data
Fetching SPS data
Fetching ABQ data
Fetching SJC data
Fetching DSM data
Fetching EWR data
Fetching SMF data
Fetching JFK data
Fetching ATL data
Fetching HSV data
Fetching ORF data
Fetching BWI data
Fetching YUM data
Fetching ORD data
Fetching DTW data
Fetching COS data
Fetching RSW data
Fetching ASE data
Fetching IAH data
Fetching LIT data
Fetching OKC data
Fetching HHH data
Fetching SNA data
Fetching MDW data
Fetching SEA data
Fetching CVG data
Fetching PVD data
Fetching RDU data
Fetching SLC data
Fetching SHV data
Fetching LAX data
Fetching ELP data
Fetchi

In [16]:
len(data.keys())

305

In [17]:
with open("airport_info.json", "w") as out:
    json.dump(data, fp=out)